## Data loading and initial scan 

1. **Import** all the libraries, functions and files necessary for the project

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from functions import *
%reload_ext autoreload

In [3]:
df_demo = pd.read_csv('data/df_final_demo.txt')
df_web_data_pt1 = pd.read_csv('data/df_final_web_data_pt_1.txt')
df_web_data_pt2 = pd.read_csv('data/df_final_web_data_pt_2.txt')
df_experiment_clients = pd.read_csv('data/df_final_experiment_clients.txt')

2. **Join the web data** 

Explore the datasets that contain the information of the detailed trace of client interactions online, which is divided into two parts.
To evaluate if it is possible to join them, with what method and if it is necessary to do some type of cleaning first.

In [4]:
explore(df_web_data_pt1)

(343141, 5)
   client_id            visitor_id                      visit_id process_step  \
0    9988021  580560515_7732621733  781255054_21935453173_531117       step_3   
1    9988021  580560515_7732621733  781255054_21935453173_531117       step_2   
2    9988021  580560515_7732621733  781255054_21935453173_531117       step_3   
3    9988021  580560515_7732621733  781255054_21935453173_531117       step_2   
4    9988021  580560515_7732621733  781255054_21935453173_531117       step_3   
5    9988021  580560515_7732621733  781255054_21935453173_531117       step_2   
6    9988021  580560515_7732621733  781255054_21935453173_531117       step_1   
7    9988021  580560515_7732621733  781255054_21935453173_531117        start   
8    8320017  39393514_33118319366  960651974_70596002104_312201      confirm   
9    8320017  39393514_33118319366  960651974_70596002104_312201       step_3   

             date_time  
0  2017-04-17 15:27:07  
1  2017-04-17 15:26:51  
2  2017-04-17 15:19:2

In [5]:
explore(df_web_data_pt2)

(412264, 5)
   client_id             visitor_id                      visit_id  \
0     763412  601952081_10457207388  397475557_40440946728_419634   
1    6019349  442094451_91531546617  154620534_35331068705_522317   
2    6019349  442094451_91531546617  154620534_35331068705_522317   
3    6019349  442094451_91531546617  154620534_35331068705_522317   
4    6019349  442094451_91531546617  154620534_35331068705_522317   
5    6019349  442094451_91531546617  154620534_35331068705_522317   
6    4726500  934350987_45569789638  467318052_88159801968_565608   
7    4726500  934350987_45569789638  467318052_88159801968_565608   
8    4726500  934350987_45569789638  467318052_88159801968_565608   
9    4726500  934350987_45569789638  467318052_88159801968_565608   

  process_step            date_time  
0      confirm  2017-06-06 08:56:00  
1      confirm  2017-06-01 11:59:27  
2       step_3  2017-06-01 11:58:48  
3       step_2  2017-06-01 11:58:08  
4       step_1  2017-06-01 11:57:58  


We use the concatenate method because we want to join the DataFrames by stacking rows. To facilitate the general analysis of the final information on the website.

In [6]:
df_web_data = pd.concat([df_web_data_pt1, df_web_data_pt2], ignore_index=True)

3. **Explore the DataFrames**


We apply a function that prints the head, the null and datatype information of the columns.

In [7]:
explore(df_demo)

(70609, 9)
   client_id  clnt_tenure_yr  clnt_tenure_mnth  clnt_age gendr  num_accts  \
0     836976             6.0              73.0      60.5     U        2.0   
1    2304905             7.0              94.0      58.0     U        2.0   
2    1439522             5.0              64.0      32.0     U        2.0   
3    1562045            16.0             198.0      49.0     M        2.0   
4    5126305            12.0             145.0      33.0     F        2.0   
5    3727881             5.0              71.0      30.5     U        2.0   
6     272934             5.0              66.0      58.5     U        2.0   
7     388801            30.0             361.0      57.5     M        5.0   
8     285619            30.0             369.0      67.5     M        2.0   
9    8198645            15.0             189.0      54.5     F        2.0   

         bal  calls_6_mnth  logons_6_mnth  
0   45105.30           6.0            9.0  
1  110860.30           6.0            9.0  
2   52467

In [8]:
explore(df_web_data)

(755405, 5)
   client_id            visitor_id                      visit_id process_step  \
0    9988021  580560515_7732621733  781255054_21935453173_531117       step_3   
1    9988021  580560515_7732621733  781255054_21935453173_531117       step_2   
2    9988021  580560515_7732621733  781255054_21935453173_531117       step_3   
3    9988021  580560515_7732621733  781255054_21935453173_531117       step_2   
4    9988021  580560515_7732621733  781255054_21935453173_531117       step_3   
5    9988021  580560515_7732621733  781255054_21935453173_531117       step_2   
6    9988021  580560515_7732621733  781255054_21935453173_531117       step_1   
7    9988021  580560515_7732621733  781255054_21935453173_531117        start   
8    8320017  39393514_33118319366  960651974_70596002104_312201      confirm   
9    8320017  39393514_33118319366  960651974_70596002104_312201       step_3   

             date_time  
0  2017-04-17 15:27:07  
1  2017-04-17 15:26:51  
2  2017-04-17 15:19:2

In [9]:
explore(df_experiment_clients)

(70609, 2)
   client_id Variation
0    9988021      Test
1    8320017      Test
2    4033851   Control
3    1982004      Test
4    9294070   Control
5    9466803      Test
6    3170143      Test
7    6651403   Control
8    5887227      Test
9    2105948   Control
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70609 entries, 0 to 70608
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  70609 non-null  int64 
 1   Variation  50500 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.1+ MB
None


## Cleaning data

### 1. Cleaning df_demo

Call the function that will remove rows with too many null values and duplicate rows, to remove what is not essential.

In [10]:
df_demo = remove_nulls_and_duplicates(df_demo, threshold=0.8)

c:\Users\almuo\Desktop\IRONHACK\PROJECT 2 AB TESTING\Vanguard-AB-Test-Analysis\functions.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.drop_duplicates(inplace=True)


Change the names of the columns of this dataframe, to make it simpler and more understandable.

In [11]:
df_demo = rename_columns_1(df_demo)

We fill in the null values by the mean in the age column. Since there is only one null value and the mean is a representative value of the data set, it will not change the results.

In [12]:
df_demo = mean_fill_missing_values(df_demo, 'client_age')

Convert the gender column to a categorical data type to facilitate analysis.

In [13]:
df_demo = convert_to_categorical(df_demo, 'gender')

### 2. Cleaning df_web_data

Convert the date_time column to a datetime format to be able to work with dates and times more easily.

In [14]:
df_web_data = convert_to_datetime(df_web_data, 'date_time')

### 3. Cleaning df_experiment_clients

We changed the name of the column 'Variation' to 'experiment_group' to avoid the use of capital letters and make it more understandable.

In [15]:
df_experiment_clients.rename(columns={'Variation': 'experiment_group'}, inplace=True)

We remove the rows with null values in the Variation column, because their value is crucial (it distinguishes between “Test” and “Control”). Therefore, we cannot fill them in, nor use them for analysis without knowing which group they belong to.

In [16]:
df_experiment_clients = df_experiment_clients.dropna(subset=['experiment_group'])

Convert the gender column to a categorical data type to facilitate analysis.

In [17]:
df_experiment_clients = convert_to_categorical(df_experiment_clients, 'experiment_group')

## Condensing web data

### 1. Extract data from the original dataframe

We extract information from the df_web_data to **create a dataframe** in which each customer has a single row. The original dataframe has many records per client, for each interaction on the web site, totaling 755405 rows. We **condens the information** to reduce the amount of redundant data. 

The result is a more manageable dataframe, where **each row represents a unique customer with its key metrics**. This structure will facilitate efficient merge with other dataframes and subsequent analysis, which are essential for our A/B testing analysis.

- **Number of visits**: count how many unique visits (visit_id) each customer made in the online process. It can be related to the efficiency of the website.

In [18]:
# Count the number of visits per client
visits_count = df_web_data.groupby('client_id')['visit_id'].nunique().reset_index()
visits_count.columns = ['client_id', 'num_visits']

- **Number of steps**: count the total number of unique steps (process_step) that each customer completed in all visits. It shows us how far each customer progressed through the process and the efficiency of the user flow.

In [19]:
# Count the number of steps for each client
steps_count = df_web_data.groupby('client_id').size().reset_index(name='num_steps')

- **Completion_status**: is a Boolean indicator that marks whether the customer reached the final step (confirm) or not. It is essential to measure the success of the process and to calculate the flow completion rate.

In [20]:
# Calculate completion rate
df_web_data['completed'] = df_web_data['process_step'] == 'confirm'
completion_rate = df_web_data.groupby('client_id')['completed'].max().reset_index()
completion_rate.columns = ['client_id', 'completion_status']

- **Total time spent**: add up the total time each customer spent in the online process, first calculating the difference between the successive steps. It could indicate difficulties in navigation or overly complex processes.

In [21]:
#Order the dataframe to calculate the difference in time
df_web_data = df_web_data.sort_values(by=['visitor_id', 'date_time'])

# Calculate time spent creating a column in the original dataframe
df_web_data['time_spent'] = df_web_data.groupby('visit_id')['date_time'].diff().shift(-1)

In [22]:
# Calculate total time spent per client
total_time_per_client = df_web_data.groupby('client_id')['time_spent'].sum().reset_index()
total_time_per_client.columns = ['client_id', 'total_time_spent']
total_time_per_client.head(10)

,client_id,total_time_spent
0,169,0 days 00:03:33
1,336,0 days 00:15:48
2,546,0 days 00:02:13
3,555,0 days 00:02:38
4,647,0 days 00:06:17
5,722,0 days 00:09:59
6,786,0 days 00:05:12
7,805,0 days 00:03:38
8,832,0 days 00:02:14
9,934,0 days 00:02:22


- **Repeated step**: boolean indicator that signals if the client repeated any step in the process. May be a sign of confusion or usability problems in the flow.

In [23]:
repeat_steps = df_web_data.groupby('client_id')['process_step'].value_counts()

# Create a dataframe with the number of times each client has repeated each step
repeat_steps_df = repeat_steps.reset_index(name='count')

# Create a new column indicating if the client has repeated each step
repeat_steps_df['repeated_step'] = repeat_steps_df['count'] > 1

# Create a new dataframe with only the clients who have repeated each step
df_repeated = repeat_steps_df.groupby('client_id')['repeated_step'].any().reset_index()


### 2. Create the condensed dataframe

Join all the information we extracted from the original dataframe(df_web_data) in df_web.

In [32]:
df_web = visits_count.copy()

df_web = df_web.merge(steps_count, on='client_id', how='left')
df_web = df_web.merge(completion_rate, on='client_id', how='left')
df_web = df_web.merge(total_time_per_client, on='client_id', how='left')
df_web = df_web.merge(df_repeated, on='client_id', how='left')

### 3. Complete the dataframe

- **Steps per visit**: Calculated by dividing the total number of steps (num_steps) by the number of visits (num_visits), it tells us how many steps on average a customer completed in their visits.

In [25]:
#Complete the information with an average of steps per visit
df_web['steps_per_visit'] = (df_web['num_steps']/df_web['num_visits']).round(2)

- **Total time in seconds**: the seconds version of the total time, that allows for more detailed analysis by switching from datatime to integer format.

In [26]:
#Complete the information with a column of the time expressed in seconds
df_web['total_time_seconds'] = df_web['total_time_spent'].dt.total_seconds().astype(int)

**Order the columns**: so that the column layout in the dataframe makes more sense in the reading order of the data.

In [27]:
columnas_ordenadas = ['client_id', 'num_visits', 'num_steps', 'steps_per_visit', 'repeated_step', 'total_time_spent', 'total_time_seconds', 'completion_status']
df_web = df_web[columnas_ordenadas]

df_web.head(10)

,client_id,num_visits,num_steps,steps_per_visit,repeated_step,total_time_spent,total_time_seconds,completion_status
0,169,1,5,5.0,False,0 days 00:03:33,213,True
1,336,1,2,2.0,True,0 days 00:15:48,948,False
2,546,1,5,5.0,False,0 days 00:02:13,133,True
3,555,1,5,5.0,False,0 days 00:02:38,158,True
4,647,1,5,5.0,False,0 days 00:06:17,377,True
5,722,1,9,9.0,True,0 days 00:09:59,599,True
6,786,1,6,6.0,True,0 days 00:05:12,312,True
7,805,3,9,3.0,True,0 days 00:03:38,218,False
8,832,1,7,7.0,True,0 days 00:02:14,134,True
9,934,1,4,4.0,True,0 days 00:02:22,142,False


## Join dataframes

In [28]:
df_complete = df_demo.merge(df_experiment_clients, on='client_id', how='left')
df_complete = df_complete.merge(df_web, on='client_id', how='left')

In [38]:
explore(df_complete)

(70595, 17)
   client_id  client_tenure_years  client_tenure_months  client_age gender  \
0     836976                  6.0                  73.0        60.5      U   
1    2304905                  7.0                  94.0        58.0      U   
2    1439522                  5.0                  64.0        32.0      U   
3    1562045                 16.0                 198.0        49.0      M   
4    5126305                 12.0                 145.0        33.0      F   
5    3727881                  5.0                  71.0        30.5      U   
6     272934                  5.0                  66.0        58.5      U   
7     388801                 30.0                 361.0        57.5      M   
8     285619                 30.0                 369.0        67.5      M   
9    8198645                 15.0                 189.0        54.5      F   

   num_accounts    balance  calls_last_6_months  logons_last_6_months  \
0           2.0   45105.30                  6.0         

¿Queremos conservar las entradas a la web de las que no tenemos datos de cliente, ni de si eran parte del experimento?